In [1]:
import pandas as pd
import numpy as np
import time
from citipy import citipy
import requests
from config import weather_api_key


In [2]:
# Create a set of random lat and long combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
longs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_longs = zip(lats, longs)

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_longs)

In [4]:
# create a list for holding the cities.
cities = []
#identify the nearest city for each lat long combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    #if the city is unique then we will at it to the cities list.
    if city not in cities:
        cities.append(city)
#Print the city count to confirm sufficient count.
len(cities)

769

In [5]:
# Starting URL for Weather Map API Call.(from module)
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
# create an empty list to hold weather data
city_data = []
#print the beginning of the loging.
print("Beginning Data Retrieval    ")
print("----------------------------")

#create counter
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        current_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": current_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval    
----------------------------
Processing Record 1 of Set 1 | ahipara
Processing Record 2 of Set 1 | port elizabeth
Processing Record 3 of Set 1 | vaini
Processing Record 4 of Set 1 | broken hill
Processing Record 5 of Set 1 | burgeo
Processing Record 6 of Set 1 | cidreira
Processing Record 7 of Set 1 | san cristobal
Processing Record 8 of Set 1 | ban nahin
Processing Record 9 of Set 1 | butaritari
Processing Record 10 of Set 1 | rikitea
Processing Record 11 of Set 1 | ihosy
Processing Record 12 of Set 1 | deer lake
Processing Record 13 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 14 of Set 1 | cuamba
Processing Record 15 of Set 1 | olafsvik
Processing Record 16 of Set 1 | pajapan
Processing Record 17 of Set 1 | vilyuysk
Processing Record 18 of Set 1 | punta arenas
Processing Record 19 of Set 1 | santiago del estero
Processing Record 20 of Set 1 | puerto escondido
Processing Record 21 of Set 1 | cabo san lucas
Processing Record 22 of

In [7]:
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ahipara,NZ,-35.1667,173.1667,63.88,74,35,11.52,scattered clouds
1,Port Elizabeth,ZA,-33.9180,25.5701,75.09,83,20,13.80,few clouds
2,Vaini,TO,-21.2000,-175.2000,73.56,100,40,1.14,fog
3,Broken Hill,AU,-31.9500,141.4333,73.11,64,83,4.61,broken clouds
4,Burgeo,CA,47.6167,-57.6152,25.47,67,88,17.72,overcast clouds
...,...,...,...,...,...,...,...,...,...
706,Chanika,TZ,-5.4167,38.0167,75.20,74,100,7.87,light rain
707,Lapeer,US,43.0514,-83.3188,26.49,58,0,10.36,clear sky
708,Novo Aripuana,BR,-5.1206,-60.3797,92.41,50,98,2.17,overcast clouds
709,Usinsk,RU,65.9939,57.5281,-6.92,91,27,10.87,scattered clouds


In [8]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")